In [1]:
! pip install python-gdcm
! pip install pylibjpeg pylibjpeg-libjpeg pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 37.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 11.7 MB/s eta 0:00:0000:0100:01


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pydicom
import os
import nibabel as nib
import SimpleITK as sitk
from tqdm import tqdm
from tensorflow import keras
from keras import layers, models
import tensorflow_io as tfio
import cv2
import warnings
warnings.filterwarnings('ignore')
from skimage.transform import resize
from tqdm import tqdm
import tensorflow_addons as tfa

In [4]:
def load_image(path):
    img = pydicom.read_file(path)
    img = img.pixel_array
    return cv2.resize(img[35:-35,35:-35], (160,160))

In [5]:
def window(img):
    img[np.where(img<0)] = 0
    img[np.where(img>1900)] = 1900
    return img/1900.0

In [6]:
def channeling(img):
#     std = img.std()
#     mean = img.mean()
#     img1 = img.copy()
#     img2 = img.copy()
    
#     img1[np.where(img1<(mean-std))] = (mean-std)
#     img1[np.where(img1>(mean))] = mean
    
#     img2[np.where(img2<(mean))] = mean
#     img2[np.where(img2>(mean+std))] = (mean+std)
    
#     img[np.where(img<(mean+std))]= (mean+std)
    
    return np.stack([img, img, img], axis=-1)

In [7]:
bb_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train_bounding_boxes.csv")
bb_df

,StudyInstanceUID,x,y,width,height,slice_number
0,1.2.826.0.1.3680043.10051,219.27715,216.71419,17.30440,20.38517,133
1,1.2.826.0.1.3680043.10051,221.56460,216.71419,17.87844,25.24362,134
2,1.2.826.0.1.3680043.10051,216.82151,221.62546,27.00959,26.37454,135
3,1.2.826.0.1.3680043.10051,214.49455,215.48637,27.92726,37.51363,136
4,1.2.826.0.1.3680043.10051,214.00000,215.48637,27.00000,43.51363,137
...,...,...,...,...,...,...
7212,1.2.826.0.1.3680043.9940,297.23186,115.53983,85.18228,66.52623,140
7213,1.2.826.0.1.3680043.9940,298.00000,117.00000,86.00000,61.00000,141
7214,1.2.826.0.1.3680043.9940,298.00000,119.00000,87.00000,58.00000,142
7215,1.2.826.0.1.3680043.9940,299.00000,120.00000,89.00000,56.00000,143


In [8]:
df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
df = df[df['patient_overall']==0]
df

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
7,1.2.826.0.1.3680043.17364,0,0,0,0,0,0,0,0
8,1.2.826.0.1.3680043.849,0,0,0,0,0,0,0,0
12,1.2.826.0.1.3680043.24946,0,0,0,0,0,0,0,0
13,1.2.826.0.1.3680043.9290,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2011,1.2.826.0.1.3680043.32263,0,0,0,0,0,0,0,0
2012,1.2.826.0.1.3680043.6398,0,0,0,0,0,0,0,0
2013,1.2.826.0.1.3680043.31621,0,0,0,0,0,0,0,0
2016,1.2.826.0.1.3680043.14341,0,0,0,0,0,0,0,0


In [9]:
path = "../input/rsna-2022-cervical-spine-fracture-detection/train_images/"

In [10]:
import random

def my_distribution(maxx):
    THRESHOLD = [0.05, 0.95]
    u = random.random()
    if u <= THRESHOLD[0]:
        return random.randint(1, maxx//5)
    elif u <= THRESHOLD[1]:
        return random.randint( (maxx//5),  ((4*maxx)//5))
    else:
        return random.randint(((4*maxx)//5), maxx)

In [26]:
np_list = np.arange(0,7217*4)
random_list = np.random.choice(np_list,7217*4,replace=False)
t = 4
n_all = t*7217
n_80 = ((t*7217)*(8))//10
deviation = n_all//1058 

def data_gen():
    for i in random_list[:n_80]:
        if i%4!=0:
            try:
                label = 0
                img = window(load_image(path+df.iloc[int(deviation)]["StudyInstanceUID"]+"/"+str(my_distribution(len(os.listdir(path+df.iloc[int(deviation)]["StudyInstanceUID"]))))+".dcm"))
                yield channeling(img),label
            except:
                continue

        else:         
            label = 1
            img = window(load_image(path+bb_df.iloc[i//t]["StudyInstanceUID"]+"/"+str(bb_df.iloc[i//t]["slice_number"])+".dcm"))
            yield channeling(img),label
        
        
def test_gen():
    for i in random_list[n_80:]:
        if i%4!=0:
            try:
                label = 0
                img = window(load_image(path+df.iloc[int(deviation)]["StudyInstanceUID"]+"/"+str(my_distribution(len(os.listdir(path+df.iloc[int(deviation)]["StudyInstanceUID"]))))+".dcm"))
                yield channeling(img),label
            except:
                continue

        else:         
            label = 1
            img = window(load_image(path+bb_df.iloc[i//t]["StudyInstanceUID"]+"/"+str(bb_df.iloc[i//t]["slice_number"])+".dcm"))
            yield channeling(img),label



In [27]:
dataset = tf.data.Dataset.from_generator(
     data_gen,
    (tf.float32, tf.int8),
    (tf.TensorShape([160,160,3]), tf.TensorShape([])))
testset = tf.data.Dataset.from_generator(
     test_gen,
    (tf.float32, tf.int8),
    (tf.TensorShape([160,160,3]), tf.TensorShape([])))

In [28]:
augment = tf.keras.Sequential([
  keras.layers.RandomFlip("horizontal_and_vertical"),
  keras.layers.RandomRotation(0.4),
  keras.layers.GaussianNoise(0.1),
  keras.layers.RandomFlip(mode="horizontal_and_vertical", seed=5),
  keras.layers.RandomContrast(0.6, seed=2),
  keras.layers.RandomZoom(
    (-0.3, 0.3),
    (-0.3, 0.3),
    fill_mode='reflect',
    interpolation='bilinear',
    seed=3)
])
    

In [29]:
dataset = dataset.map(lambda x, y: (augment(x), y), 
              num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.batch(30)
dataset = dataset.prefetch(2)


testset = testset.batch(40)
testset = testset.prefetch(2)

In [30]:
res = tf.keras.applications.EfficientNetB7(
    include_top=False,
    weights="imagenet",
    pooling='avg',
    input_shape=(160,160,3),
)

inputs = keras.Input((160, 160, 3), name = "input_1")
features = res(inputs)
x = keras.layers.Flatten()(features)
output = keras.layers.Dense(1 ,activation = 'sigmoid', name="co")(x)
                                               
model = keras.models.Model(inputs=inputs, outputs=output)

258088960/258076736 [==============================] - 13s 0us/step


In [32]:
opt = tfa.optimizers.AdamW(
    weight_decay=2e-4,
    learning_rate = 0.0009,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'AdamW',
)
loss = tf.keras.losses.BinaryCrossentropy(
    label_smoothing=0.1, 
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_crossentropy'
)
metrics = [tf.keras.metrics.AUC()]
model.compile(optimizer=opt,
              loss=loss,
                metrics=metrics)

In [ ]:
history = model.fit(dataset, validation_data = testset, epochs=10, class_weight={1:3, 0:1})

Epoch 1/10
770/770 [==============================] - 629s 772ms/step - loss: 0.3883 - auc_2: 0.9931 - val_loss: 1.7689 - val_auc_2: 0.5603
Epoch 2/10
770/770 [==============================] - 585s 760ms/step - loss: 0.3355 - auc_2: 0.9985 - val_loss: 2.4572 - val_auc_2: 0.5000
Epoch 3/10
770/770 [==============================] - 585s 759ms/step - loss: 0.3323 - auc_2: 0.9987 - val_loss: 1.5735 - val_auc_2: 0.7372
Epoch 4/10
770/770 [==============================] - 585s 759ms/step - loss: 0.3279 - auc_2: 0.9989 - val_loss: 2.0267 - val_auc_2: 0.5318
Epoch 5/10
770/770 [==============================] - 584s 758ms/step - loss: 0.3285 - auc_2: 0.9988 - val_loss: 1.6926 - val_auc_2: 0.8353
Epoch 6/10
770/770 [==============================] - 583s 758ms/step - loss: 0.3314 - auc_2: 0.9988 - val_loss: 1.8462 - val_auc_2: 0.6817
Epoch 7/10
770/770 [==============================] - 583s 757ms/step - loss: 0.3293 - auc_2: 0.9989 - val_loss: 1.8379 - val_auc_2: 0.7018
Epoch 8/10
111/770 [

In [ ]:
model.predict(testset.take(2))